# Домашнее задание к лекции «Продвинутый pandas 2»

# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [1]:
import pandas as pd

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [46]:
user_min_time = ratings.groupby('userId').agg({'rating':'count', 'timestamp':'min'}).reset_index()
user_min_time['date_min'] = pd.to_datetime(user_min_time.timestamp, unit='s')
user_min_time['year_min'] = user_min_time.date_min.dt.year
morecount = (user_min_time[user_min_time.rating>100])
user_morecount = morecount.filter(items = ['userId', 'rating','date_min','year_min'])
user_morecount.head()

,userId,rating,date_min,year_min
3,4,204,2000-02-05 19:25:14,2000
7,8,116,2006-07-31 23:42:20,2006
14,15,1700,2001-08-16 04:47:19,2001
16,17,363,2005-09-23 09:43:07,2005
18,19,423,1997-02-06 00:48:11,1997


In [47]:
user_max_time = ratings.groupby('userId').agg({'timestamp':'max'}).reset_index()
user_max_time['date_max'] = pd.to_datetime(user_max_time.timestamp, unit='s')
user_max_time['year_max'] = user_max_time.date_max.dt.year
user_max_time_f = user_max_time.filter(items = ['userId','date_max','year_max'])
user_max_time_f.head()

,userId,date_max,year_max
0,1,2009-12-14 02:53:25,2009
1,2,1996-06-21 11:24:06,1996
2,3,2011-02-28 22:39:47,2011
3,4,2000-02-08 03:57:54,2000
4,5,2006-11-12 23:45:45,2006


In [48]:
a = user_morecount.merge(user_max_time_f, on = 'userId', how = 'left')
a.head()

,userId,rating,date_min,year_min,date_max,year_max
0,4,204,2000-02-05 19:25:14,2000,2000-02-08 03:57:54,2000
1,8,116,2006-07-31 23:42:20,2006,2006-08-01 23:22:07,2006
2,15,1700,2001-08-16 04:47:19,2001,2016-07-24 03:25:35,2016
3,17,363,2005-09-23 09:43:07,2005,2005-09-23 11:57:20,2005
4,19,423,1997-02-06 00:48:11,1997,1997-02-06 02:16:13,1997


In [52]:
a['time_life'] = a.date_max - a.date_min
a.sort_values(by = 'time_life',ascending = False).head()


,userId,rating,date_min,year_min,date_max,year_max,delta,time_life
136,380,1063,2000-02-01 01:05:25,2000,2016-06-05 19:59:46,2016,5969 days 18:54:21,5969 days 18:54:21
208,547,2391,2000-11-21 03:25:09,2000,2016-10-16 03:14:04,2016,5807 days 23:48:55,5807 days 23:48:55
157,427,294,1999-10-07 21:43:07,1999,2014-11-13 18:04:03,2014,5515 days 20:20:56,5515 days 20:20:56
2,15,1700,2001-08-16 04:47:19,2001,2016-07-24 03:25:35,2016,5455 days 22:38:16,5455 days 22:38:16
243,624,1735,2002-04-18 09:57:46,2002,2016-10-16 11:12:53,2016,5295 days 01:15:07,5295 days 01:15:07


# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [53]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [54]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [55]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [56]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [60]:
clien_no_adress = client_base.filter(items = ['client_id'])
clien_no_adress


,client_id
0,111
1,112
2,113
3,114
4,115
5,116
6,117
7,118


In [65]:
table_1 = clien_no_adress.merge(rzd, on = 'client_id', how = 'outer').merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')
table_1

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [66]:
table_2 = client_base.merge(rzd, on = 'client_id', how = 'outer').merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')
table_2

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

Ответ: Если в обеих таблицах сходятся только ID, то надо написать функцию, которая нумерует каждого пользователя отдельно в его покупках. Так же и пронумировать для каждого пользователя отдельно набор координат в его заходах. И тогда их можно юудет объединить по этим номерам.Т е, например, user_ID = 11 заходил 3 раза и 3 раза ему зафиксировали координаты. Следовательно объединить первый заход с первыми координатами и так все заходы со всеми коордлинатами. Далее переходить к следующему user_ID и проделывать то же самое.

Из координат можно получить адрес. Объединив адрес с покупками пользователя можно потом сегментировать пользователей по регионам и основным потребностям. направлять рекламные предложения сегментированно, а не всем подряд.